##Importing Libraries and Loading Data

In [1]:
import pandas as pd
from prophet import Prophet
from matplotlib import pyplot
from matplotlib.pyplot import figure
from sklearn.metrics import mean_absolute_error
import plotly.express as px
import plotly.graph_objects as go

In [2]:
df=pd.read_csv('/content/Data.csv')
df.head()

,States,Regions,latitude,longitude,Dates,Usage
0,Punjab,NR,31.519974,75.980003,02/01/2019 00:00:00,119.9
1,Haryana,NR,28.450006,77.019991,02/01/2019 00:00:00,130.3
2,Rajasthan,NR,26.449999,74.639981,02/01/2019 00:00:00,234.1
3,Delhi,NR,28.669993,77.230004,02/01/2019 00:00:00,85.8
4,UP,NR,27.599981,78.050006,02/01/2019 00:00:00,313.9


In [3]:
df.shape

(16599, 6)

In [4]:
df.dtypes

States        object
Regions       object
latitude     float64
longitude    float64
Dates         object
Usage        float64
dtype: object

##Preprocessing and Changing Data for Prophet Model

In [5]:
df['Dates'] = pd.to_datetime(df['Dates'], dayfirst=True)
df.head()

,States,Regions,latitude,longitude,Dates,Usage
0,Punjab,NR,31.519974,75.980003,2019-01-02,119.9
1,Haryana,NR,28.450006,77.019991,2019-01-02,130.3
2,Rajasthan,NR,26.449999,74.639981,2019-01-02,234.1
3,Delhi,NR,28.669993,77.230004,2019-01-02,85.8
4,UP,NR,27.599981,78.050006,2019-01-02,313.9


In [6]:
df=df[['Dates','Usage']]
df=df.groupby('Dates',as_index=False).sum()
print(df.head())

       Dates   Usage
0 2019-01-02  3373.4
1 2019-01-03  3403.7
2 2019-01-04  3304.1
3 2019-01-05  3308.9
4 2019-01-06  3316.9


In [7]:
fig = px.line(df, x='Dates', y='Usage')
fig.show()

In [8]:
df.columns = ['ds','y']

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnwm49v_p/ultuvir1.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnwm49v_p/s5ps4007.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=82056', 'data', 'file=/tmp/tmpnwm49v_p/ultuvir1.json', 'init=/tmp/tmpnwm49v_p/s5ps4007.json', 'output', 'file=/tmp/tmpnwm49v_p/prophet_modelwqwk5w02/prophet_model-20240724051643.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
05:16:43 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
05:16:43 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


##Fitting Prophet Model

In [9]:
model=Prophet()
model.fit(df)
model.component_modes

{'additive': ['weekly',
  'additive_terms',
  'extra_regressors_additive',
  'holidays'],
 'multiplicative': ['multiplicative_terms', 'extra_regressors_multiplicative']}

In [10]:
future_dates = model.make_future_dataframe(periods=365,freq='d',include_history=True)
future_dates.shape
future_dates.head()

,ds
0,2019-01-02
1,2019-01-03
2,2019-01-04
3,2019-01-05
4,2019-01-06


In [11]:
prediction=model.predict(future_dates)
prediction.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2019-01-02,3327.516745,2788.515835,3839.749852,3327.516745,3327.516745,8.216745,8.216745,8.216745,8.216745,8.216745,8.216745,0.0,0.0,0.0,3335.733490
1,2019-01-03,3328.376375,2834.186360,3820.589973,3328.376375,3328.376375,11.538358,11.538358,11.538358,11.538358,11.538358,11.538358,0.0,0.0,0.0,3339.914733
2,2019-01-04,3329.236005,2832.178660,3833.184746,3329.236005,3329.236005,12.411808,12.411808,12.411808,12.411808,12.411808,12.411808,0.0,0.0,0.0,3341.647813
3,2019-01-05,3330.095635,2767.967299,3823.308105,3330.095635,3330.095635,-38.390234,-38.390234,-38.390234,-38.390234,-38.390234,-38.390234,0.0,0.0,0.0,3291.705400
4,2019-01-06,3330.955265,2778.965734,3845.926903,3330.955265,3330.955265,-28.306774,-28.306774,-28.306774,-28.306774,-28.306774,-28.306774,0.0,0.0,0.0,3302.648491


In [12]:
trace_open = go.Scatter(
    x = prediction["ds"],
    y = prediction["yhat"],
    mode = 'lines',
    name="Forecast"
)
trace_high = go.Scatter(
    x = prediction["ds"],
    y = prediction["yhat_upper"],
    mode = 'lines',
    fill = "tonexty",
    line = {"color": "#57b8ff"},
    name="Higher uncertainty interval"
)
trace_low = go.Scatter(
    x = prediction["ds"],
    y = prediction["yhat_lower"],
    mode = 'lines',
    fill = "tonexty",
    line = {"color": "#57b8ff"},
    name="Lower uncertainty interval"
)
trace_close = go.Scatter(
    x = df["ds"],
    y = df["y"],
    name="Data values"
)

data = [trace_open,trace_high,trace_low,trace_close]
layout = go.Layout(title="Power consumption forecasting")

fig = go.Figure(data=data)
fig.show()

In [13]:
fig = go.Figure([go.Scatter(x=df['ds'], y=df['y'],mode='lines',
                    name='Actual')])
fig.add_trace(go.Scatter(x=prediction['ds'], y=prediction['yhat'],
                   mode='lines+markers',
                    name='predicted'))
fig.show()

In [14]:
y_true = df['y'].values
y_pred = prediction['yhat'][:498].values

mae = mean_absolute_error(y_true, y_pred)
print('MAE: %.3f' % mae)

MAE: 280.519


##Optimizing the Model

In [15]:
model1=Prophet(daily_seasonality=True).add_seasonality(name='yearly',period=365,fourier_order=70)
model1.fit(df)

INFO:prophet:Found custom seasonality named 'yearly', disabling built-in 'yearly' seasonality.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnwm49v_p/5auyhsxi.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnwm49v_p/ydjmq3hj.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=15605', 'data', 'file=/tmp/tmpnwm49v_p/5auyhsxi.json', 'init=/tmp/tmpnwm49v_p/ydjmq3hj.json', 'output', 'file=/tmp/tmpnwm49v_p/prophet_modelw5_mk6bk/prophet_model-20240724051644.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
05:16:44 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
05:16:45 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [16]:
model1.component_modes

{'additive': ['yearly',
  'weekly',
  'daily',
  'additive_terms',
  'extra_regressors_additive',
  'holidays'],
 'multiplicative': ['multiplicative_terms', 'extra_regressors_multiplicative']}

In [17]:
future_dates1=model1.make_future_dataframe(periods=365)
prediction1=model1.predict(future_dates1)

In [18]:
y_true = df['y'].values
y_pred = prediction1['yhat'][:498].values
mae = mean_absolute_error(y_true, y_pred)
print('MAE: %.3f' % mae)

MAE: 140.042


In [19]:
import plotly.graph_objects as go
fig = go.Figure([go.Scatter(x=df['ds'], y=df['y'],mode='lines',
                    name='Actual')])

fig.add_trace(go.Scatter(x=prediction1['ds'], y=prediction1['yhat'],
                   mode='lines+markers',
                    name='predicted'))

fig.show()